In [20]:
import re, time
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Dropout
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.utils import np_utils

In [21]:
data = pd.read_csv('newstitle_gbp_xau.csv')
data.head()

,timestamp,title,gbp,xau
0,1167580800,Italy puts froth back into cappuccino Patients...,1.959094,324.767450
1,1167667200,Pork soup handouts not racist - French judge Y...,1.973442,324.524002
2,1167753600,Iraqi PM rules out second term Zimbabwe poised...,1.949440,322.762609
3,1167840000,Senior soldier condemns state of forces housin...,1.944289,322.798264
4,1167926400,Three of Pinochet family cleared News in Brief...,1.929921,314.234051


In [22]:
vectorizer = TfidfVectorizer(min_df=10, ngram_range=(2, 5))
X = vectorizer.fit_transform(data['title'].tolist())
X

<3670x1776 sparse matrix of type '<class 'numpy.float64'>'
	with 40097 stored elements in Compressed Sparse Row format>

In [23]:
gbp = data.as_matrix(columns=['gbp']).reshape(-1)
xau = data.as_matrix(columns=['xau']).reshape(-1)

In [24]:
gbp = preprocessing.normalize(gbp)
xau = preprocessing.normalize(xau)

/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [25]:
gbp = np.gradient(gbp.reshape(-1))
xau = np.gradient(xau.reshape(-1))
gbp = np.gradient(gbp.reshape(-1))
xau = np.gradient(xau.reshape(-1))

In [26]:
# def Discretization(Y, offset = 1e-4):
#     Y[np.where(Y >= offset)] = 1
#     Y[np.where(Y <= -offset)] = -1
#     return Y

In [27]:
d = True 
if d == True:
    gbp = np.where(gbp > 0, 1, 0)
    xau = np.where(xau > 0, 1, 0)
    Y = np.where((gbp-xau) == 0, 1, 0)
    print(np.bincount(Y))
# else:
#     gbp = Discretization(gbp, offset = 1e-4)
#     xau = Discretization(xau, offset = 1e-4)
#     Y = gbp - xau
#     Y[np.where(Y == 0)] = 1
#     Y[np.where((Y == 2) & (Y == -2))] = -1
#     #Y[np.where((Y != 1) & (Y != -1))] = 0
#     Y = Y.astype('int64')
gbp.shape

[2077 1593]


(3670,)

In [28]:
num_classes = 2
Y = np_utils.to_categorical(Y, num_classes)
Y

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [29]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
# X_train = X_train.toarray()
# X_test = X_test.toarray()


In [30]:
# model = LogisticRegression(fit_intercept=True,
#                            class_weight = 'balanced',
#                            n_jobs=-1,
#                            max_iter=200, 
#                            solver='lbfgs',  
#                            verbose=1, 
#                            tol=1e-10)
# model.fit(X_train, y_train)
# model.score(X_test, y_test)

In [31]:
# recall = cross_val_score(model, X, Y, cv=5, n_jobs = -1, scoring='recall')
# precision = cross_val_score(model, X, Y, cv=5, n_jobs = -1, scoring='precision')
# print('precision:',np.mean(precision))
# print('recall:', np.mean(recall))

In [32]:
# model.fit(X_train, y_train)
# print(confusion_matrix(y_test, model.predict(X_test), labels=[-1,1]))
# target_names = ['-','+']
# print(classification_report(y_test, model.predict(X_test), target_names=target_names))

In [33]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    history = model.fit(X_train, y_train, 
                        epochs=1,
                        batch_size=128,
                        verbose = 0)
    score = model.evaluate(X_test, y_test, batch_size=734)[1]
    time.sleep(3)
    print('valid acc :%.4f'%(score))
    return score

In [34]:
def create_model():
    model = Sequential()
    model.add(Dense(2, activation='softmax', input_shape=(X.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(200, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(2000, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(20000, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='softmax'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='RMSprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    #model.summary()
    return model

In [35]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
accurcy = []
all_X = X.toarray()
for train, test in skf.split(np.ones(3670), np.ones(3670)):
        print ("Running Fold")
        model = None
        model = create_model()
        accurcy.append(train_and_evaluate_model(model, all_X[train], Y[train], all_X[test], Y[test]))
        y_true = np.argmax(Y[test],1)
        y_pred = np.argmax(model.predict(all_X[test]), 1)
        print(confusion_matrix(y_true, y_pred, labels=[0,1]))
        target_names = ['-','+']
        print(classification_report(y_true, y_pred, target_names=target_names))
        del model
        
print('final', np.mean(accurcy))

Running Fold
734/734 [==============================] - 0s
valid acc :0.5463
[[401   0]
 [333   0]]
             precision    recall  f1-score   support

          -       0.55      1.00      0.71       401
          +       0.00      0.00      0.00       333

avg / total       0.30      0.55      0.39       734

Running Fold


/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


734/734 [==============================] - 0s
valid acc :0.5531
[[406   0]
 [328   0]]
             precision    recall  f1-score   support

          -       0.55      1.00      0.71       406
          +       0.00      0.00      0.00       328

avg / total       0.31      0.55      0.39       734

Running Fold
734/734 [==============================] - 0s
valid acc :0.5613
[[412   0]
 [322   0]]
             precision    recall  f1-score   support

          -       0.56      1.00      0.72       412
          +       0.00      0.00      0.00       322

avg / total       0.32      0.56      0.40       734

Running Fold
734/734 [==============================] - 0s
valid acc :0.5790
[[425   0]
 [309   0]]
             precision    recall  f1-score   support

          -       0.58      1.00      0.73       425
          +       0.00      0.00      0.00       309

avg / total       0.34      0.58      0.42       734

Running Fold


ResourceExhaustedError: OOM when allocating tensor with shape[2000,20000]
	 [[Node: Square_159 = Square[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients_12/dense_80/MatMul_grad/MatMul_1)]]

Caused by op 'Square_159', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/lib/python3.6/site-packages/tornado/ioloop.py", line 888, in start
    h

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
# X_train = X_train.toarray()
# X_test = X_test.toarray()
# model = create_model()
# history = model.fit(X_train, y_train, 
#                     epochs=3,
#                     batch_size=128,
#                     verbose = 1)
